In [ ]:
import os
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
import torch.nn.init as init
from torch.autograd import Variable
from dataloader import load_cifar

In [ ]:
# Hyperparams
EPOCHS  =  3
BATCHSIZE  =  64
LR  =  0.01
MOMENTUM  =  0.9
N_CLASSES  =  10
GPU  =  True
BATCH_SIZE  =  32
RESNET_FEATURES  =  2048
BATCHES_GPU  =  40

torch.backends.cudnn.benchmark=True
torch.cuda.current_device()
torch.cuda._initialized = True

### Load dataset

In [ ]:
x_train, x_test, y_train, y_test = load_cifar(channel_first=True)
# Torch-specific
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int64)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

### Define model

In [ ]:
class SymbolModule(nn.Module):
    def __init__(self, n_classes=N_CLASSES):
        super(SymbolModule, self).__init__()
        self.conv1 = nn.Conv2d(3, 50, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(50, 50, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(50, 100, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(100, 100, kernel_size=3, padding=1)
        # feature map size is 8*8 by pooling
        self.fc1 = nn.Linear(100*8*8, 512)
        self.fc2 = nn.Linear(512, n_classes)

    def forward(self, x):
        # PyTorch requires a flag for training in dropout
        x = self.conv2(F.relu(self.conv1(x)))
        x = F.relu(F.max_pool2d(x, kernel_size=2, stride=2))
        x = F.dropout(x, 0.25, training=self.training)

        x = self.conv4(F.relu(self.conv3(x)))
        x = F.relu(F.max_pool2d(x, kernel_size=2, stride=2))
        x = F.dropout(x, 0.25, training=self.training)

        x = x.view(-1, 100*8*8)   # reshape Variable
        x = F.dropout(F.relu(self.fc1(x)), 0.5, training=self.training)
        return self.fc2(x)
    
def init_model(m, lr=LR, momentum=MOMENTUM):
    # This criterion combines nn.LogSoftmax() and nn.NLLLoss() in one single class
    opt = optim.SGD(m.parameters(), lr, momentum)
    criterion = nn.CrossEntropyLoss()
    return opt, criterion

### Init model

In [ ]:
sym = SymbolModule()
sym.cuda() # CUDA!
optimizer, criterion = init_model(sym)

### Main training loop

In [ ]:
def shuffle_data(X, y):
    s = np.arange(len(X))
    np.random.shuffle(s)
    X = X[s]
    y = y[s]
    return X, y

def yield_mb(X, y, batchsize=64, shuffle=False):
    if shuffle:
        X, y = shuffle_data(X, y)
    # Only complete batches are submitted
    for i in range(len(X) // batchsize):
        yield X[i * batchsize:(i + 1) * batchsize], y[i * batchsize:(i + 1) * batchsize]

sym.train() # Sets training = True  
for j in range(EPOCHS):
    for data, target in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        # Get samples
        data = Variable(torch.FloatTensor(data).cuda())
        target = Variable(torch.LongTensor(target).cuda())
        # Init
        optimizer.zero_grad()
        # Forwards
        output = sym(data)
        # Loss
        loss = criterion(output, target)
        # Back-prop
        loss.backward()
        optimizer.step()
    # Log
    print("EPOCHS:{},LOSS:{}".format(j,loss))

### Main evaluation loop

In [ ]:
sym.eval() # Sets training = False
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, target in yield_mb(x_test, y_test, BATCHSIZE):
    # Get samples
    data = Variable(torch.FloatTensor(data).cuda())
    # Forwards
    output = sym(data)
    pred = output.data.max(1)[1].cpu().numpy().squeeze()
    # Collect results
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = pred
    c += 1

print("Accuracy: ", 1.*sum(y_guess == y_truth)/len(y_guess))